# Calculating the Levelized Cost of Charging 
EIA reports and [The Utility Rate Database](https://openei.org/wiki/Utility_Rate_Database) were used to estimate the annual cost of electricity for EV charging loads at the rate-, utility-, and state-levels. For residential TOU rates (URDB), it is assumed that the consumer will always charge at the lowest possible rate (off-peak). In determining the cost of electricity for residential charging, rates with demand charges are not included as they would require typical household energy profiles that vary greatly on a case-by-case basis. For estimating the cost of electricity for commercial use (e.g. DCFC stations), typical energy profiles (electricity demand scenarios) were generated to estimate fixed, demand, and energy charges for a variety of rate structures (TOU, tier, seasonal, demand, etc.).

In [1]:
import sys
import logging
import numpy as np
import pandas as pd

sys.path.append(os.path.join('..',''))
import config
import lcoc.urdb as urdb
import lcoc.afdc as afdc
import lcoc.processing as proc
import os

### Load Data:

In [2]:
urdb_path = config.URDB_PATH
print(config.URDB_PATH)
db = urdb.DatabaseRates(urdb_path)
print("Total:", db.rate_data.shape)
print("Res:", db.res_rate_data.shape)
print("Com:", db.com_rate_data.shape)

..\data\urdb\usurdb_20190724.csv
Total: (48556, 577)
Res: (10972, 577)
Com: (31236, 577)


### Preprocessing:

In [3]:
#filter expired rates
db.filter_stale_rates(industry='residential')
print("Res:", db.res_rate_data.shape)

db.filter_stale_rates(industry='commercial')
print("Com:", db.com_rate_data.shape)

Res: (6094, 577)
Com: (17886, 577)


In [4]:
#classify rates by is_tier, is_seasonal, is_TOU, is_ev-specific (residential only)
ev_rate_words_filepath = '../filters/urdb_res_ev_specific_rate_words.txt'

db.classify_rate_structures(industry='residential', 
                            ev_rate_words_file=ev_rate_words_filepath)

db.classify_rate_structures(industry='commercial')

#standardize units of reporting for commercial rates
db.com_rate_preprocessing()

In [5]:
filters_path = '..\\filters\\'

#filter demand rates (residential only)
db.filter_demand_rates(industry='residential') 

# filter commercial rates missing critical fields to approx the cost of electricity
db.additional_com_rate_filters()

#filter rates containing certain phrases in filters/
db.filter_on_phrases(industry='residential', filters_path=filters_path)
db.filter_on_phrases(industry='commercial', filters_path=filters_path)

#combine base rate + adjusted rate
db.combine_rates(industry='residential')
db.combine_rates(industry='commercial')

#filter null rates
db.filter_null_rates(industry='residential')
db.filter_null_rates(industry='commercial')

print("Res:", db.res_rate_data.shape)
print("Com:", db.com_rate_data.shape)

Res: (4729, 639)
Com: (6020, 638)


In [6]:
#res rate structure breakdown
db.generate_classification_tree_values(industry='residential')

{'demand': 0,
 'no_demand': 4729,
 'demand/tier': 0,
 'demand/fixed': 0,
 'no_demand/tier': 1981,
 'no_demand/fixed': 2748,
 'demand/tier/seasonal': 0,
 'demand/tier/no_seasonal': 0,
 'demand/fixed/seasonal': 0,
 'demand/fixed/no_seasonal': 0,
 'no_demand/tier/seasonal': 814,
 'no_demand/tier/no_seasonal': 1167,
 'no_demand/fixed/seasonal': 525,
 'no_demand/fixed/no_seasonal': 2223,
 'demand/tier/seasonal/tou': 0,
 'demand/tier/seasonal/no_tou': 0,
 'demand/tier/no_seasonal/tou': 0,
 'demand/tier/no_seasonal/no_tou': 0,
 'demand/fixed/seasonal/tou': 0,
 'demand/fixed/seasonal/no_tou': 0,
 'demand/fixed/no_seasonal/tou': 0,
 'demand/fixed/no_seasonal/no_tou': 0,
 'no_demand/tier/seasonal/tou': 52,
 'no_demand/tier/seasonal/no_tou': 762,
 'no_demand/tier/no_seasonal/tou': 19,
 'no_demand/tier/no_seasonal/no_tou': 1148,
 'no_demand/fixed/seasonal/tou': 258,
 'no_demand/fixed/seasonal/no_tou': 267,
 'no_demand/fixed/no_seasonal/tou': 313,
 'no_demand/fixed/no_seasonal/no_tou': 1910}

In [7]:
#com rate structure breakdown
db.generate_classification_tree_values(industry='commercial') 

{'demand': 4159,
 'no_demand': 1861,
 'demand/tier': 1251,
 'demand/fixed': 2908,
 'no_demand/tier': 617,
 'no_demand/fixed': 1244,
 'demand/tier/seasonal': 149,
 'demand/tier/no_seasonal': 1102,
 'demand/fixed/seasonal': 502,
 'demand/fixed/no_seasonal': 2406,
 'no_demand/tier/seasonal': 191,
 'no_demand/tier/no_seasonal': 426,
 'no_demand/fixed/seasonal': 223,
 'no_demand/fixed/no_seasonal': 1021,
 'demand/tier/seasonal/tou': 16,
 'demand/tier/seasonal/no_tou': 133,
 'demand/tier/no_seasonal/tou': 13,
 'demand/tier/no_seasonal/no_tou': 1089,
 'demand/fixed/seasonal/tou': 218,
 'demand/fixed/seasonal/no_tou': 284,
 'demand/fixed/no_seasonal/tou': 248,
 'demand/fixed/no_seasonal/no_tou': 2158,
 'no_demand/tier/seasonal/tou': 3,
 'no_demand/tier/seasonal/no_tou': 188,
 'no_demand/tier/no_seasonal/tou': 4,
 'no_demand/tier/no_seasonal/no_tou': 422,
 'no_demand/fixed/seasonal/tou': 75,
 'no_demand/fixed/seasonal/no_tou': 148,
 'no_demand/fixed/no_seasonal/tou': 128,
 'no_demand/fixed/no_s

### Residential cost of electricity, LCOC:

In [8]:
%%time
#calculate annual electricity cost (rates)
outpath='..\\outputs\\cost-of-electricity\\urdb-res-rates\\'
db.calculate_annual_energy_cost_residential(outpath)

Complete, 4723 rates included.
Wall time: 2min 5s


In [9]:
#calculate annual electricity cost (utility-level)
urdb_rates_file = '..\\outputs\\cost-of-electricity\\urdb-res-rates\\res_rates.csv'
eia_cw_file = config.EIAID_TO_UTILITY_CW_PATH
eia_utils_file = config.EIA_RES_PATH
outpath = '..\\outputs\\cost-of-electricity\\res-utilities\\'

df = proc.res_rates_to_utils(scenario='baseline',
                             urdb_rates_file=urdb_rates_file,
                             eia_cw_file=eia_cw_file,
                             eia_utils_file=eia_utils_file,
                             outpath=outpath)

Complete, 2986 utitilies represented (309 TOU rates used).


In [24]:
#calculate annual electricity cost (state-level)
utils_file = '..\\outputs\\cost-of-electricity\\res-utilities\\res_utils.csv' #lower_bnd_res_utils.csv, upper_bnd_res_utils.csv
outfile = '..\\outputs\\cost-of-electricity\\res-states\\res_states_baseline.csv' #res_states_lower_bnd.csv, res_states_upper_bnd.csv
proc.res_utils_to_state(utils_file=utils_file, outfile=outfile)

Complete, national cost of electricity is $0.08/kWh.


In [26]:
vmt = np.array([13065] * 3 + [12582] * 3 + [11432] * 4 + [7812] * 5) #NHTS 2017 (Table 22)
lifetime_vmt = vmt.sum()
aavmt = lifetime_vmt / 15
low_aavmt = 100000 / 15
high_aavmt = 200000 / 15

In [29]:
#calculate levelized cost of charging (state-level) BEV
coe_file = '..\\outputs\\cost-of-electricity\\res-states\\res_states_baseline.csv' #res_states_lower_bnd.csv, res_states_upper_bnd.csv, res_states_baseline.csv

print("BEV:")
# os.mkdir(os.path.join('..', 'outputs', 'cost-of-charging', 'residential'))
proc.calculate_state_residential_lcoc(coe_file=coe_file,
                                      fixed_costs_path = '..\\data\\fixed-costs\\residential\\',
                                      veh_kwh_per_100miles = 28.33, #source: EPA
                                      aavmt = aavmt, #average annual VMT
                                      fraction_residential_charging = 0.81, #source: EPRI report
                                      fraction_home_l1_charging = 0.16, #source: EPRI report (baseline - 0.16)
                                      outfile = '..\\outputs\\cost-of-charging\\residential\\res_states_baseline.csv')

BEV:
LCOC calculation complete, national LCOC (residential) is $0.12/kWh


In [30]:
#calculate levelized cost of charging (state-level) PHEV
coe_file = '..\\outputs\\cost-of-electricity\\res-states\\res_states_baseline.csv' #res_states_lower_bnd.csv, res_states_upper_bnd.csv, res_states_baseline.csv
phev_charge_depl_perc = 0.76 #2019 EPA Automotive Trends Report - Table E.1, GM Volt
aavmt_phev = aavmt * phev_charge_depl_perc

print("PHEV:")
proc.calculate_state_residential_lcoc(coe_file=coe_file,
                                      fixed_costs_path = '..\\data\\fixed-costs\\residential\\',
                                      veh_kwh_per_100miles = 28.33, #EPA
                                      aavmt = aavmt_phev,
                                      fraction_residential_charging = 0.81, #source: EPRI report
                                      fraction_home_l1_charging = 0.5, #source: EPRI report (baseline=0.5)
                                      outfile = '..\\outputs\\cost-of-charging\\residential\\res_states_phev_baseline.csv')

PHEV:
LCOC calculation complete, national LCOC (residential) is $0.11/kWh


### Workplace/Public-L2 LCOC:

In [33]:
#calculate levelized cost of charging (state-level)
coe_path = config.EIA_COM_PATH

outpath = os.path.join('..', 'outputs', 'cost-of-charging', 'workplace-public-l2')
if not os.path.exists(outpath):
    os.mkdir(outpath)
proc.calculate_state_workplace_public_l2_lcoc(coe_path = coe_path,
                                              fixed_costs_file = '..\\data\\fixed-costs\\workplace-public-l2\\com_level2.txt',
                                              equip_utilization_kwh_per_day = 30,
                                              outpath = '..\\outputs\\cost-of-charging\\workplace-public-l2\\work_pub_l2_states_baseline.csv')

LCOC calculation complete, national LCOC (workplace/pub-L2) is $0.15/kWh


### DCFC cost of electricity:
**Attention: Paths must be modified to work from nbs dir**

In [34]:
%%time
#WARNING - Takes ~4 days to complete on laptop!
# db.calculate_annual_cost_dcfc(log_lvl=1) #0=WARNING, 1=INFO, 2=DEBUG

Wall time: 0 ns


In [35]:
#calculate annual electricity cost (utility-level)
proc.dcfc_rates_to_utils()

Utility-level results generated for p1.
Utility-level results generated for p2.
Utility-level results generated for p3.
Utility-level results generated for p4.


In [37]:
#AFDC
stations = afdc.DCFastChargingLocator() #GET DCFC stations from AFDC
stations.join_county_geoid(us_counties_gdf_file = os.path.join('..','data','gis','2019_counties','cb_2019_us_county_500k','cb_2019_us_county_500k.shp')) #join to county (spatial)
stations.aggregate_counties_to_csv() #aggregate to county-lvl, output to .csv

Complete, 5757 stations in 1104 counties.


In [38]:
#calculate annual electricity cost (state-level)
proc.dcfc_county_to_state()

State-level results generated for p1.
State-level results generated for p2.
State-level results generated for p3.
State-level results generated for p4.


### DCFC LCOC:
**note** - Values in *`outputs/cost-of-charging/dcfc/baseline_state_profiles.csv`* must be input from the EVI-FAST Excel tool before this step can be completed.

In [39]:
#produce single DCFC LCOC from 4 profiles
# proc.combine_dcfc_profiles_into_single_lcoc()

### Single LCOC:
Combine residential, workplace/public-L2, and DCFC LCOC estimates into a single state-level LCOC.

In [40]:
#calculate combined (res, work, pub-L2, DCFC) LCOC (state-level)
print("BEV:")
proc.combine_res_work_dcfc_lcoc(res_wgt=0.81,#0.81, #0.45 (free workplace)
                                wrk_wgt=0.14, #0.5 (free workplace)
                                dcfc_wgt=0.05,
                                res_lcoc_file='..\\outputs\\cost-of-charging\\residential\\res_states_baseline.csv',
                                wrk_lcoc_file = '..\\outputs\\cost-of-charging\\workplace-public-l2\\work_pub_l2_states_baseline.csv',
                                dcfc_lcoc_file = '..\\outputs\\cost-of-charging\\dcfc\\dcfc_states_baseline.csv',
                                outfile='..\\outputs\\cost-of-charging\\comb\\comb_states_baseline.csv')

BEV:
Combined LCOC calculation complete, national LCOC is $0.13/kWh


In [ ]:
print("PHEV:")
proc.combine_res_work_dcfc_lcoc(res_wgt=0.81, #0.81
                                wrk_wgt=0.19, #0.19
                                dcfc_wgt=0,
                                res_lcoc_file = '..\\outputs\\cost-of-charging\\residential\\res_states_phev_baseline.csv',
                                wrk_lcoc_file = '..\\outputs\\cost-of-charging\\workplace-public-l2\\work_pub_l2_states_baseline.csv',
                                outfile = '..\\outputs\\cost-of-charging\\comb\\comb_states_phev_baseline.csv')


Perform left join on comb_states_lower_bnd w/ comb_states_lower_bnd_res_equip:  
  
(note: some states did not list a TOU rate in the URDB and, by default, are omitted from the table. This operation adds these states to the comb_states_lower_bnd tables by reporting the comb_states_lower_bnd_res_equip values)

In [ ]:
#BEV
bev_lower_bnd_df = pd.read_csv('..\\outputs\\cost-of-charging\\comb\\comb_states_lower_bnd.csv')
bev_lower_bnd_df.index = bev_lower_bnd_df['state']
bev_lower_bnd_df.drop(columns='state', inplace=True)

bev_low_bnd_res_equip_df = pd.read_csv('..\\outputs\\cost-of-charging\\comb\\comb_states_lower_bnd_res_equip.csv')
bev_low_bnd_res_equip_df.index = bev_low_bnd_res_equip_df['state']
bev_low_bnd_res_equip_df.drop(columns='state', inplace=True)

bev_low_bnd_res_equip_df.update(bev_lower_bnd_df)
new_bev_lower_bnd_df = bev_low_bnd_res_equip_df.reset_index()
new_bev_lower_bnd_df.to_csv('..\\outputs\\cost-of-charging\\comb\\comb_states_lower_bnd.csv')

#PHEV
phev_lower_bnd_df = pd.read_csv('..\\outputs\\cost-of-charging\\comb\\comb_states_phev_lower_bnd.csv')
phev_lower_bnd_df.index = phev_lower_bnd_df['state']
phev_lower_bnd_df.drop(columns='state', inplace=True)

phev_low_bnd_res_equip_df = pd.read_csv('..\\outputs\\cost-of-charging\\comb\\comb_states_phev_lower_bnd_res_equip.csv')
phev_low_bnd_res_equip_df.index = phev_low_bnd_res_equip_df['state']
phev_low_bnd_res_equip_df.drop(columns='state', inplace=True)

phev_low_bnd_res_equip_df.update(phev_lower_bnd_df)
new_phev_lower_bnd_df = phev_low_bnd_res_equip_df.reset_index()
new_phev_lower_bnd_df.to_csv('..\\outputs\\cost-of-charging\\comb\\comb_states_phev_lower_bnd.csv')